syntehsize fast ones?

Known bits.
https://pypy.org/posts/2024/08/toy-knownbits.html
https://arxiv.org/pdf/2105.05398

In [ ]:
from kdrag.all import *
Interval = kd.Struct("Interval", ("start", smt.IntSort()), ("end", smt.IntSort()))#), pred=lambda i: i.start <= i.end)
x,y,z = smt.Ints("x y z")
i1,i2 = smt.Consts("i1 i2", Interval)
elem = kd.define("elem", [x,i1], smt.And(i1.start <= x, x <= i1.end))
to_set = kd.define("to_set", [i1], smt.Lambda([x], smt.And(i1.start <= x, x <= i1.end)))
int_add = kd.notation.add.define([i1,i2], Interval(i1.start + i2.start, i1.end + i2.end))
kd.notation.le.define([i1,i2], smt.And(i1.start <= i2.start, i1.end <= i2.end))
le_set = kd.prove(smt.ForALl([i1,i2], kd.Implies(i1 <= i2, smt.ForAll([x], elem(x,i1), elem(x,i2)))))

correct = smt.QForAll([i1,i2], smt.QForAll([x,y], elem(x,i1), elem(y,i2), elem(x + y, i1 + i2)))
best = kd.QForAll([i1,i2,i3], kd.QForAll([x], elem(x,i1), elem(x,i2), elem(x,i3),) i1 + i2 <= i3)

smt.Implies(i.start <= i.end , from_set(to_set(i)) == i) # ? What about
smt.Implies()



In [ ]:
from dataclasses import dataclass

@dataclass(eq=False)
class KnownBits:
    ones : int
    unknowns : int

    def __post_init__(self):
        if isinstance(self.ones, int):
            assert self.is_well_formed()
    def is_well_formed(self):
        return self.ones & self.unknowns == 0
    @staticmethod
    def from_constant(const : int):
        return KnownBits(const, 0)
    def is_constant(self):
        return self.unknowns == 0
    @property
    def knowns(self):
        return ~self.unknowns
    @property
    def zeros(self):
        return self.knowns & ~self.ones
    def contains(self, value : int):
        return value & self.knowns == self.ones
    def __invert__(self):
        return KnownBits(self.zeros, self.unknowns)
    def __and__(self, other):
        ones = self.ones & other.ones # known ones
        knowns = self.zeros | other.zeros | ones
        return KnownBits(ones, ~knowns)
    #def join(self, other):
    #    return KnownBits( &) , self.unknowns | other.unknowns | (self.ones ^ other.ones))

~KnownBits.from_constant(5)
~5

-6

In [ ]:
from Enum import Enum, auto
class TriState(Enum):
    TRUE = auto()
    FALSE = auto()
    UNKNOWN = auto()
